### Police departments to coordinates

In [ ]:
# import pandas as pd
# from geopy.geocoders import Nominatim
# from tqdm import tqdm

# data = pd.read_csv('data.csv', delimiter=';')

# geolocator = Nominatim(user_agent="geoapiExercises")

# def get_lat_long(query):
#     try:
#         # location = geolocator.geocode(f"{agency}, {state}, USA")
#         location = geolocator.geocode(query)

#         if location:
            
#             return location.raw
#             return location.latitude, location.longitude
#         else:
#             return None, None
#     except:
#         return None, None

In [ ]:
from dotenv import load_dotenv

load_dotenv('.env') 

API_KEY = os.getenv('API_KEY')

In [1]:
import pandas as pd
import requests
from tqdm import tqdm

data = pd.read_csv('data.csv', delimiter=';')

def get_lat_long(state, agency):
    url = f'https://api.opencagedata.com/geocode/v1/json?q={agency}, {state}, USA&key={API_KEY}'
    try:
        response = requests.get(url).json()
        if response['results']:
            location = response['results'][0]['geometry']
            return location['lat'], location['lng']
        else:
            return None, None
    except:
        return None, None

In [2]:
get_lat_long("AL, USA", "GUNTERSVILLE POLICE DEPT") # test

(32.75041, -86.75026)

### Run this if the data_with_lat_long does not exist

In [4]:
data['Latitude'] = None
data['Longitude'] = None

unique_combinations = data[['State', 'Agency Name']].drop_duplicates()
lat_long_dict = {}

for _, row in tqdm(unique_combinations.iterrows(), total=unique_combinations.shape[0]):
    state = row['State']
    agency = row['Agency Name']
    lat, long = get_lat_long(state, agency)
    lat_long_dict[(state, agency)] = (lat, long)
for i, row in data.iterrows():
    
    state = row['State']
    agency = row['Agency Name']
    data.at[i, 'Latitude'] = lat_long_dict[(state, agency)][0]
    data.at[i, 'Longitude'] = lat_long_dict[(state, agency)][1]
    
data.to_csv('data_with_lat_long.csv', index=False)

  1%|          | 32/5389 [00:14<43:19,  2.06it/s]

### Run it again

In [3]:
data = pd.read_csv('data_with_lat_long.csv')

missing_lat_long = data[(data['Latitude'].isnull()) | (data['Longitude'].isnull())]

unique_missing_combinations = missing_lat_long[['State', 'Agency Name']].drop_duplicates()

lat_long_dict = {}

for _, row in tqdm(unique_missing_combinations.iterrows(), total=unique_missing_combinations.shape[0]):
    state = row['State']
    agency = row['Agency Name']
    lat, long = get_lat_long(state, agency)
    lat_long_dict[(state, agency)] = (lat, long)

for i, row in missing_lat_long.iterrows():
    state = row['State']
    agency = row['Agency Name']
    if lat_long_dict[(state, agency)][0] is not None and lat_long_dict[(state, agency)][1] is not None:
        missing_lat_long.at[i, 'Latitude'] = lat_long_dict[(state, agency)][0]
        missing_lat_long.at[i, 'Longitude'] = lat_long_dict[(state, agency)][1]

data.update(missing_lat_long)
data.to_csv('data_with_lat_long_1.csv', index=False)

100%|██████████| 526/526 [03:36<00:00,  2.43it/s]


In [5]:
import pandas as pd

df = pd.read_csv('data_with_lat_long_1.csv')

json_data = df.to_json(orient='records')

with open('data.json', 'w') as file:
    file.write(json_data)
